In [1]:
import sys
sys.path.insert(1, '/home/lruffati/Documents/0.University/0.SistemiElettorali/')
import src

from src.Metaclasses import *
import src.GlobalVars as GlobalVars
import random

In [2]:
class HubClass:
    def __init__(self, *args):
        self.instances_dict = {}
        self.subs_relations = {}
    
    def get_subdivisions(self, sup, sub_name):
        """
        sup: o un'istanza di una geoEnt, o una tupla (nomeClasse, nomeIstanza)
        sub_name: nome classe target

        return: list of names (stringhe)
        """
        print("Get subdivisions: ", sup, sub_name)
        typ = None
        inst = sup
        if type(sup) == tuple:
            typ = sup[0]
            inst = self.get_instance(typ, sup[1])
        else:
            typ = sup.type
            name = sup.name
        
        var_name = self.subs_relations[typ][sub_name]
        
        print("getattr ", inst, var_name)
        return getattr(inst, var_name)
    
    def get_superdivision(self, sub, sup_name):
        """
        sub: o un'istanza o una tuple
        sup_name: nome classe target

        return nome istanza target
        """
        pass

    def add_instance(self, class_name, inst_name, instance):
        instances = self.instances_dict.get(class_name, {})
        instances[inst_name] = instance
        self.instances_dict[class_name] = instances
    
    def get_instance(self, class_name, inst_name):
        print("Get inst: ", class_name, inst_name)
        return self.instances_dict[class_name][inst_name]

    def add_subdiv(self, sup_type, sub_type, var_name):
        """
        Records that instances of sup_type have instances of sub_type 
        stored in self.var_name
        """
        print("Adding sub: ",sup_type, sub_type, var_name)
        sup_dict = self.subs_relations.get(sup_type, {})
        sup_dict[sub_type] = var_name
        self.subs_relations[sup_type]= sup_dict

src.GlobalVars.Hub = HubClass()

In [3]:
def print_dict(dic, indent=0):
    if type(dic) == list:
        for i in dic:
            print_dict(i, indent)
        return
    if type(dic) != dict:
        print(indent*"--", dic)
        return
    
    for k, d in dic.items():
        print(indent*"--", k, ":")
        print_dict(d, indent+1)

In [4]:
config = {
 'lane': {'plurinominale': {'first_input': 'kwarg',
                            'info_name': 'Circoscrizione',
                            'node_type': 'node',
                            'operations': [{'collect_type': 'lista',
                                            'corrector': 'commons.correct',
                                            'ideal_distribution': '$'}],
                            'sub_level': 'Plurinominale'}},
 'lanes_propose': {'elettori': {'distribution': ['Elettore', 'Seggi'],
                                'info': ['RestoCircoCoal',
                                         'RestoCircoCoalUsato'],
                                'source': {'args': [{'source': {'columns': ['Elettore -> Party',
                                                                            'Voti -> Votes'],
                                                                'name': 'self.eleggibili',
                                                                'type': 'fun'}},
                                                    {'source': {'name': 'self.subs_plurinominali_seggi',
                                                                'type': 'fun'}}],
                                           'columns': ['Party -> Elettore',
                                                       'Seats -> Seggi',
                                                       'Remainder -> RestoCircoCoal',
                                                       'RemainderUsed -> RestoCircoCoalUsato'],
                                           'name': 'commons.hondt',
                                           'type': 'fun'}},
                   'liste': {'distribution': ['Lista', 'Seggi'],
                             'info': ['RestoCircoLista',
                                      'RestoCircoListaUsato'],
                             'source': {'kwargs': {'seggi': {'source': {'name': 'constraints',
                                                                        'type': 'kwarg'}}},
                                        'name': 'self.ripartisci_seggi',
                                        'type': 'fun'}}},
 'metaclasses': ['superdivision', 'totals', 'lanes', 'external'],
 'subdivisions': {'plurinominali': {'functions': [{'name': 'seggi',
                                                   'source': {'name': 'self.seggi',
                                                              'type': 'att'}},
                                                  {'name': 'liste',
                                                   'source': {'totals': 'lista'}},
                                                  {'name': 'coalizioni',
                                                   'source': {'totals': 'coalizione'}}],
                                    'type': 'Plurinominale'}},
 'totals': {'coalizione': {'keys': ['Coalizione'],
                           'ops': {'Voti': 'sum'},
                           'source': {'name': 'self.subs_plurinominali_coalizioni',
                                      'type': 'fun'},
                           'type': 'aggregate'},
            'lista': {'keys': ['Lista'],
                      'ops': {'Voti': 'sum'},
                      'source': {'name': 'self.subs_plurinominali_liste',
                                 'type': 'fun'},
                      'type': 'aggregate'}},
 'totals_support': {'eleggibili': {'args': [{'source': {'args': ['coal_eletta'],
                                                        'columns': ['Coalizione -> Elettore',
                                                                    'Voti'],
                                                        'totals': 'coalizione'},
                                             'type': 'scalar'},
                                            {'source': {'args': ['eletta_sola'],
                                                        'columns': ['Lista -> Elettore',
                                                                    'Voti'],
                                                        'totals': 'lista'},
                                             'type': 'scalar'}],
                                   'function': 'commons.concat',
                                   'type': 'combine'},
                    'elettore_lista': {'ops': [{'column': 'Lista',
                                                'column_type': 'Partito',
                                                'replace_name': 'Elettore',
                                                'source': {'attr': ['elettore'],
                                                           'name': 'commons.exec_fun',
                                                           'type': 'fun'},
                                                'type': 'column'}],
                                       'source': {'totals': 'lista'},
                                       'type': 'transform'},
                    'ripartisci_seggi': {'args': [{'source': {'args': ['eletta'],
                                                              'rename': {'Lista': 'Party',
                                                                         'Voti': 'Votes'},
                                                              'totals': 'lista'},
                                                   'type': 'dataframe'},
                                                  {'source': {'columns': ['Lista -> Party',
                                                                          'Elettore'],
                                                              'name': 'self.elettore_lista',
                                                              'type': 'fun'},
                                                   'type': 'dataframe'},
                                                  {'source': {'columns': ['Elettore',
                                                                          'Seggi -> seats'],
                                                              'name': 'seggi',
                                                              'type': 'kwarg'},
                                                   'type': 'series'}],
                                         'function': 'commons.hondt',
                                         'keys': ['Elettore'],
                                         'merge_keys': ['Party'],
                                         'rename': {'Party': 'Lista',
                                                    'Remainder': 'RestoCircoLista',
                                                    'RemainderUsed': 'RestoCircoListaUsato',
                                                    'Seats': 'Seggi'},
                                         'type': 'combine'}}}

In [5]:
#conf_aft_superdiv = src.Metaclasses.superdivision.parse_conf(config)
conf_aft_totals = src.Metaclasses.totals.parse_conf(config)
conf_aft_lanes = src.Metaclasses.lanes.parse_conf(conf_aft_totals)
conf_aft_external = src.Metaclasses.external.parse_conf(conf_aft_lanes)
conf_fin = src.Metaclasses.sources_parse.source_parse(conf_aft_external)

Parsing conf:  {'lane': {'plurinominale': {'first_input': 'kwarg', 'info_name': 'Circoscrizione', 'node_type': 'node', 'operations': [{'collect_type': 'lista', 'corrector': 'commons.correct', 'ideal_distribution': '$'}], 'sub_level': 'Plurinominale'}}, 'lanes_propose': {'elettori': {'distribution': ['Elettore', 'Seggi'], 'info': ['RestoCircoCoal', 'RestoCircoCoalUsato'], 'source': {'args': [{'source': {'columns': ['Elettore -> Party', 'Voti -> Votes'], 'name': 'self.eleggibili', 'type': 'fun'}}, {'source': {'name': 'self.subs_plurinominali_seggi', 'type': 'fun'}}], 'columns': ['Party -> Elettore', 'Seats -> Seggi', 'Remainder -> RestoCircoCoal', 'RemainderUsed -> RestoCircoCoalUsato'], 'name': 'commons.hondt', 'type': 'fun'}}, 'liste': {'distribution': ['Lista', 'Seggi'], 'info': ['RestoCircoLista', 'RestoCircoListaUsato'], 'source': {'kwargs': {'seggi': {'source': {'name': 'constraints', 'type': 'kwarg'}}}, 'name': 'self.ripartisci_seggi', 'type': 'fun'}}}, 'metaclasses': ['superdivis

In [6]:
class cleanup(type):
    def __new__(mcs, *args, **kwargs):
        print("Conf cleanup: ", args, kwargs)
        if '__init__' in args[2]:
            o_init = args[2].get('__init__')
            def n__init__(self, name, **kwargs):
                print("Init cleanup preesistente")
                src.GlobalVars.Hub.add_instance(args[0], name, self)
                self.name = name
                return o_init(self, [], **kwargs)
            args[2]['__init__'] = n__init__
        else:
            def n_init__(self, name, **kwargs):
                print("init cleanup nuovo")
                self.name = name
                src.GlobalVars.Hub.add_instance(args[0], name, self)
            
            args[2]['__init__'] = n_init__
        
        args[2]['type'] = args[0]
        
        return super().__new__(mcs, *args)

comb = type("Metaclasse", (superdivision, totals, lanes, external, cleanup), {})

In [7]:
Circoscrizione = comb("Circoscrizione", (),{}, **conf_fin)

Superdivision configuration:  {'plurinominali': {'functions': [{'name': 'seggi', 'source': <function source_parse.<locals>.function_returned at 0x7fae33053a60>}, {'name': 'liste', 'source': <function source_parse.<locals>.function_returned at 0x7fae33050670>}, {'name': 'coalizioni', 'source': <function source_parse.<locals>.function_returned at 0x7fae330505e0>}], 'type': 'Plurinominale'}}
Adding sub:  Circoscrizione Plurinominale plurinominali
After superdivision:  ('Circoscrizione', (), {'subs_plurinominali_seggi': <function superdivision.generate_accessor.<locals>.accessor at 0x7fae32ff35e0>, 'subs_plurinominali_liste': <function superdivision.generate_accessor.<locals>.accessor at 0x7fae32ff30d0>, 'subs_plurinominali_coalizioni': <function superdivision.generate_accessor.<locals>.accessor at 0x7fae32ff33a0>}) {'lane': {'plurinominale': {'first_input': 'kwarg', 'info_name': 'Circoscrizione', 'node_type': 'node', 'operations': [{'collect_type': 'lista', 'corrector': 'commons.correct',

In [8]:
c = Circoscrizione("ciao", plurinominali=['a','b','c'])

Init cleanup preesistente
Init operations of external: 
Adding  ['a', 'b', 'c']  come  plurinominali


In [9]:
src.GlobalVars.Hub.instances_dict

{'Circoscrizione': {'ciao': <__main__.Circoscrizione at 0x7fae33009dc0>}}

In [10]:
import pandas as pd

In [11]:
class PolEnt:
    def __init__(self, name):
        self.name = name
        src.GlobalVars.Hub.add_instance("PolEnt", name, self)

class Plurinominale:
    def __init__(self, name, seggi):
        self.seggi = seggi
        self.name = name
        src.GlobalVars.Hub.add_instance("Plurinominale", name, self)
    def totals(self, *a, **k):
        print("Totals pluri: ",a, k)
        return pd.DataFrame({'Coalizione':['a','b'],'Voti':[self.seggi, self.seggi+2]})

p1 = Plurinominale('a',5)
p2 = Plurinominale('b',4)
p3 = Plurinominale('c',3)

In [12]:
conf_fin['totals']['lista']['source']({'self':c})

ret_fun_parse: []
Locals prima di cercare la funzione:  {'self': <__main__.Circoscrizione object at 0x7fae33009dc0>}
Cerco funzione:  self.subs_plurinominali_liste
<bound method superdivision.generate_accessor.<locals>.accessor of <__main__.Circoscrizione object at 0x7fae33009dc0>>
Get subdivisions:  <__main__.Circoscrizione object at 0x7fae33009dc0> Plurinominale
getattr  <__main__.Circoscrizione object at 0x7fae33009dc0> plurinominali
Get inst:  Plurinominale a
Get inst:  Plurinominale b
Get inst:  Plurinominale c
sup_accessor:  [<__main__.Plurinominale object at 0x7fae32fef370>, <__main__.Plurinominale object at 0x7fae32fef8b0>, <__main__.Plurinominale object at 0x7fae32fef3a0>]
source:  <function source_parse.<locals>.function_returned at 0x7fae33050670>
ret_fun_parse: [<function source_parse.<locals>.simple_res at 0x7fae33050700>]
Locals prima di cercare la funzione:  {'self': <__main__.Plurinominale object at 0x7fae32fef370>}
Cerco funzione:  self.totals
<bound method Plurinomina

,Coalizione,Voti
0,a,5
1,b,7
2,a,4
3,b,6
4,a,3
5,b,5


In [13]:
c.subs_plurinominali_seggi()

Get subdivisions:  <__main__.Circoscrizione object at 0x7fae33009dc0> Plurinominale
getattr  <__main__.Circoscrizione object at 0x7fae33009dc0> plurinominali
Get inst:  Plurinominale a
Get inst:  Plurinominale b
Get inst:  Plurinominale c
sup_accessor:  [<__main__.Plurinominale object at 0x7fae32fef370>, <__main__.Plurinominale object at 0x7fae32fef8b0>, <__main__.Plurinominale object at 0x7fae32fef3a0>]
source:  <function source_parse.<locals>.function_returned at 0x7fae33053a60>
Locals prima di trovare l'attributo:  {'self': <__main__.Plurinominale object at 0x7fae32fef370>}
Locals prima di trovare l'attributo:  {'self': <__main__.Plurinominale object at 0x7fae32fef8b0>}
Locals prima di trovare l'attributo:  {'self': <__main__.Plurinominale object at 0x7fae32fef3a0>}


12

In [ ]:
c.totals('lista')

In [ ]:
c.subs_plurinominali_seggi()

In [ ]:
conf_fin['subdivisions']['plurinominali']['functions'][0]['source']

In [ ]:
c.propose('elettori')

In [14]:
c.totals('coalizione', 'ciao')

Total supports locs (332):  {'self': <__main__.Circoscrizione object at 0x7fae33009dc0>}
ret_fun_parse: []
Locals prima di cercare la funzione:  {'self': <__main__.Circoscrizione object at 0x7fae33009dc0>}
Cerco funzione:  self.subs_plurinominali_coalizioni
<bound method superdivision.generate_accessor.<locals>.accessor of <__main__.Circoscrizione object at 0x7fae33009dc0>>
Get subdivisions:  <__main__.Circoscrizione object at 0x7fae33009dc0> Plurinominale
getattr  <__main__.Circoscrizione object at 0x7fae33009dc0> plurinominali
Get inst:  Plurinominale a
Get inst:  Plurinominale b
Get inst:  Plurinominale c
sup_accessor:  [<__main__.Plurinominale object at 0x7fae32fef370>, <__main__.Plurinominale object at 0x7fae32fef8b0>, <__main__.Plurinominale object at 0x7fae32fef3a0>]
source:  <function source_parse.<locals>.function_returned at 0x7fae330505e0>
ret_fun_parse: [<function source_parse.<locals>.simple_res at 0x7fae33050430>]
Locals prima di cercare la funzione:  {'self': <__main__.P

KeyError: ('PolEnt', 'occurred at index 0')